In [1]:
;; A ideia e fazer um DSL com Horn clauses ond, dado um banco de conehcimento contendo fatos e regras e, possivel inferir se um Simbolo novo e um fato
;; ex1 : fatos = (A B), regras = (A and B -> c), temos que : infer(C) = True, ja que A and B -> C, entao os fatos agora sao : (A B C)
;; ex2 : fatos = (A B), regras = (A or K -> C), (C and B -> D),temos que : infer(D) = True, ja que A or K -> C : fatos = (A B C)  : C and B -> D : fatos = (A B C D)
;; ex3 : fatos = (A), regras = (A and K -> C), temos que : infer(C) = False, ja que K nao e um fato 
;; NOTA : Essa implementacao so e valida para AND, OR ou Fato Simples : A and B / A or B / A

In [2]:
;; Criando a Knowlege Base : lista com regras : (premissas . conclusao) e fatos : (Fato1 Fato2 ...)
(define (create-kb rules facts)
  (let ((kb-rules rules)
        (kb-facts facts))
    (lambda (part)
      (case part
        ((rules) kb-rules)
        ((facts) kb-facts)
        (else (error "ERRO" part))))))

In [3]:
;; Adiciona uma regra na kb (cria uma nova kb com essa nova regra inclusa)
(define (add-rule kb premise conclusion)
  (let* ((rules (kb 'rules))
         (new-rules (cons (cons premise conclusion) rules)))
    (create-kb new-rules (kb 'facts))))

;; mesma ideia mas para fatos
(define (add-fact kb fact)
  (let ((facts (kb 'facts)))
    (create-kb (kb 'rules) (if (member fact facts) facts (cons fact facts)))))


In [4]:
;; auxiliares
;; auxiliar pro or
(define (any pred lst)
  (cond ((null? lst) #f)
        ((pred (car lst)) #t)
        (else (any pred (cdr lst)))))

;; auxiliar pro and
(define (every pred lst)
  (cond ((null? lst) #t)
        ((pred (car lst)) (every pred (cdr lst)))
        (else #f)))


(define (filter pred lst)
  (cond ((null? lst) '())
        ((pred (car lst)) (cons (car lst) (filter pred (cdr lst))))
        (else (filter pred (cdr lst)))))

In [5]:
;; infer-loop : Para um dado goal, checar se e fato ou se e possivel deriva-lo diretamente apartir das regras, se um novo fato foi derivado -> faz mais uma iteracao
;; considerando esse novo fato, continua ate : (1) achar o goal ou (2) ter uma iterecao em que nenhum fato novo for derivado
(define (infer-loop2 kb goal)
  (let loop ((known (kb 'facts)))
    ;; check goal
    (if (member goal known)
        #t
        ;; achar a nova lista de fatos
        (let* ((rules (kb 'rules))
               (applicable
                (filter
                 (lambda (r)
                   (let ((prem (car r)))
                     (cond
                       ;; caso or -> usa any pra ver se existe uma premissa que e fato
                       ((and (pair? prem) (eq? (car prem) 'or))
                        (any (lambda (p) (member p known)) (cdr prem)))
                       ;; caso and -> usa every pra ver se todas as premissas sao fato
                       ((and (pair? prem) (eq? (car prem) 'and))
                        (every (lambda (p) (member p known)) (cdr prem)))
                       ;; se for uma lista (A B ...) -> trata como and
                       ((list? prem)
                        (every (lambda (p) (member p known)) prem))
                       ;; se for um atomo -> checa se e fato
                       (else (member prem known)))))
                 rules))
               (conclusions (map cdr applicable))
               (to-add (filter (lambda (c) (not (member c known))) conclusions))
               (new-known (append known to-add)))
          (if (equal? new-known known)
              #f
              (loop new-known))))))

In [6]:
;; teste sem os macros:

;; factos A, B
(define kb0 (create-kb '() '(A B)))

;; add rule: (and A B) => C
(define kb1 (add-rule kb0 '(A B) 'C))

;; add rule: (or X Y) => Z  (Z if X or Y)
(define kb2 (add-rule kb1 '(or X Y) 'Z))

;; add rule: (C) => D
(define kb3 (add-rule kb2 '(C) 'D))

;; tests
(display "infer C?  ==> ") (display (infer-loop2 kb3 'C)) (newline)
(display "infer D?  ==> ") (display (infer-loop2 kb3 'D)) (newline)
(display "infer Z?  ==> ") (display (infer-loop2 kb3 'Z)) (newline)
(display "infer Q?  ==> ") (display (infer-loop2 kb3 'Q)) (newline)

infer C?  ==> #t
infer D?  ==> #t
infer Z?  ==> #f
infer Q?  ==> #f


In [10]:

;; apply-to-facts : aplica uma funcao aos fatos (kb -> (fact-list -> X) -> X)
(define (apply-to-facts kb f)
  (let ((facts (kb 'facts)))
    (f facts)))

;; Exemplo de uso:
;; contar fatos
(define (count-facts lst) (length lst))
(display (apply-to-facts kb3 count-facts)) (newline)

;; verificar se um fato existe (passando um predicado)
(display (apply-to-facts kb3 (lambda (fs) (member 'A fs)))) (newline)

;; map sobre fatos (transformação)
(display (apply-to-facts kb3 (lambda (fs) (map (lambda (x) (string-append "fact-" (symbol->string x))) fs)))) (newline)


2
(A B)
(fact-A fact-B)


In [8]:
;; Definindo a Sintaxe :


(define-syntax define-facts
  (syntax-rules ()
    ((_ (facts ...))
     (set! kb (create-kb (kb 'rules) '(facts ...))))))


(define-syntax define-rule
  (syntax-rules (=> and or)
    ((_ (=> (and p1 p2 ...) conclusion))
     (set! kb (add-rule kb '(and p1 p2 ...) 'conclusion)))
    ((_ (=> (or p1 p2 ...) conclusion))
     (set! kb (add-rule kb '(or p1 p2 ...) 'conclusion)))
    ((_ (=> premise conclusion))
     (set! kb (add-rule kb '(premise) 'conclusion)))))


(define-syntax infer
  (syntax-rules ()
    ((_ goal)
     (infer-loop2 kb 'goal))))


In [9]:
;; testando a DSL completa :
;; Reset base
(define kb (create-kb '() '()))

;; Fatos
(define-facts (A))

;; Regras
(define-rule (=> (or A B) C))  ; C é verdadeiro se A ou B for verdadeiro
(define-rule (=> (and C B) D)) ; D só se ambos C e B forem verdadeiros
(define-rule (=> C E))         ; E depende de C

;; Testes
(display "Inferindo C: ") (display (infer C)) (newline)
(display "Inferindo D: ") (display (infer D)) (newline)
(display "Inferindo E: ") (display (infer E)) (newline)


Inferindo C: #t
Inferindo D: #f
Inferindo E: #t
